# Federated Semi-Supervised Learning with Inter-Client Consistency

## Abstract

提出了一种新的方法显著提高Federated Semi-Supervised Learning性能（FSSL)，主要采用了两种方法：

1. inter-cleint consistency loss

2. decomposition of the parameters into parameters for labeled and unlabeled data

### Introduction

用户的数据无法收集，用户的数据往往没有标签，根据数据的分布形式分成两种情况讨论：

![fsel-process](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/federated%20learning/fl-2/fsel-process.png?raw=true)

+ Standard Scenario

标准场景下有标签的数据和无标签的数据都分布在用户客户端上，模型训练完全在用户客户端进行。中心节点没有任何的数据，只负责聚合和反馈模型的参数。

+ Disjoint Scenario

分离场景下客户端只有无标签的数据，带标签的数据集中在中心节点上。此时客户端负责无标签数据的训练，中心节点负责带标签数据的训练，同时负责聚合和反馈模型参数。


## Federated Semi-Supervised Learning

### Preliminaries

+ Semi-Supervised Learning

  半监督学习的数据无标签数据集比例远大于带标签数据（>1:9）,$\mathcal{S} =  \{x_i,y_i\}^S_{i=1}$, $\mathcal{U} =  \{u_i\}^\mathcal{U}_{i=1}$ 带标数据集分别为带标签数据集和无标签数据集, 一般而言 $\|S\| \ll \|U\|$ ,损失相应的分带标签损失 $\ell_s(\theta)$ 和 $\ell_u(\theta)$ 为两部分，最终的损失为两者的加和: $\ell_{final}( \theta ) =  \ell_s(\theta) + \ell_u(\theta)$

+ Federated Learning

  将半监督学习应用于联邦学习，假设$G$为全局模型，$\mathcal{L}=\{l_k\}^K_{k=1}$是K个客户端的局部模型，相应的数据集$\mathcal{D}$也由K个子数据集$\mathcal{D}^{l_k}=\{x^{l_k}_i, y^{k_k}_i\}^{N^{l_k}}_{i=1}$组成，分别分布在各个客户端。在每轮训练中，全局模型G随机的选择局部模型，并聚合学习到的模型参数，应用于下一轮训练。

#### Federated Semi-Supervised Learning

+ Standard Scenario

  和传统的联邦学习一样，在本地训练带标数据集和无标数据集，分别利用$\ell_{final}( \theta ) =  \ell_s(\theta) + \ell_u(\theta)$更新本地模型，中心节点收集模型权重，聚合后更新到本地模型。

+ Disjoint Scenario

  由于带标数据和无标数据分别分布在中心节点和客户端，中心节点每轮先使用带标数据$\mathcal{S}^G$最小化损失$\ell_s(\theta^G)$进行监督学习，训练全局模型G，然后将全局模型权重$\theta^G$广播到客户端。然后激活的客户端在本地使用损失$\ell_u(\theta^{l_a})$训练无标数据$\mathcal{U}^{l_a}$。

### Federated Matching

![fsel-fedmatch-process](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/federated%20learning/fl-2/fsel-fedmatch-process.png?raw=true)

+ Inter_client Consistency Loss

  consistency regularization 作为半监督学习中一种重要的正则方法，利用了这样一种假设：对应相同图片施加不同的扰动，模型得到的预测应该是相似的。本文为FSSL引入了一种新的inter-client consistency loss：
  $$
  \sum\limits_{j = 1}^H\textbf{KL}[p^*_{\theta^{h_j}}(\textbf{y}|\textbf{u}) || p_{\theta^l}(\textbf{y}|\textbf{u}) ]
  $$
  

  其中$p^*_{\theta^h(y|x)}$是中心节点根据置信度选择的代理，帮助生成样本的人造标签，在客户端不参与训练。同时使用了数据层面的一直性正则，最终的一致性正则项$\Phi(\cdot)$如下：
  $$
  \Phi(\cdot) = CE(\hat{y}||p_{\theta^l}(\textbf{y}|\pi(\textbf{u})) + \sum\limits^{H}_{j=1}\textbf{KL}[p^*_{\theta^{h_j}}(\textbf{y}|\textbf{u}) || p_{\theta^l}  (\textbf{y} | \textbf{u})    ]
  $$
  其中$\pi(\textbf{u}))$表示对无标签数据$\textbf{u}$的随机数据增强，$\hat{\textbf{y}}$表示人造的标签：
  $$
  \hat{\textbf{y}} = \textbf{Max}( \mathbb{1} (p^*_{\theta^l}  (\textbf{y|u}))  +  \sum\limits^H_{j=1}1(p^*_{\theta^{h_j}} (\textbf{y|u})   )
  $$
  其中$1(\cdot)$表示使用softmax值得到的one-hot标签，$\textbf{Max}(\cdot)$输出具有符合度最高的one-hot标签。当生成人造标签时，抛掉置信度小于阈值$\tau$的人造标签。

#### Parameter Decomposition for Disjoint Learning

​		标准的半监督学习同时学习带标和无标数据，并共享相同的模型权重，然而这会导致模型忘记已经学习过的带标数据。通过将模型参数$\theta$分解为监督学习$\sigma$和无监督学习$\psi$两个部分，$\theta = \sigma + \psi$。

​		当对带标数据进行学习时，固定$\psi$不变，对$\sigma$进行学习，最小化如下交叉熵损失函数：
$$
minimize \mathcal{L}_s(\sigma) = \lambda_s\textbf{CE}(\textbf{y}, p_{\sigma}+ \psi^*(\textbf{y}| \widetilde{\pi}(\textbf{x}))
$$
​		其中$\textbf{x}$和$\textbf{y}$是带标签的数据，$\widetilde{\pi}(\cdot)$表示随机翻转和裁剪数据增强，

​		当对无标数据进行学习时，固定$\sigma$不变，对$\psi$进行学习，最小化如下一致性损失函数：
$$
minimize \mathcal{L}_u(\psi) = \lambda_u\Phi_{\theta^* }+ \psi(\cdot) + \lambda_{L_2}||\sigma^* - \psi ||^2_2 + \lambda_{L_1}||\psi||_1
$$
​		其中对$\psi$施加$L_1$和$L_2$正则项，让$\psi$变得稀疏，从而避免$\sigma$从已经学习到知识漂移的太远，同时稀疏的参数提高的通信效率。

#### Reliability-based Aggregatioin

​		由于不是每个客户端在无标数据上学习到的模型都有可信度，因此使用基于可行度评估的方法强化已经学习到的可信知识、弱化不可信知识的影响。
$$
\Gamma(\theta^{l_{1:A}}) = \frac{Acc^{l_a}}{TotalAcc^{l_{1:A}}}\sum\limits^A_{a=1}(\theta^{l_a})
$$
​		其中$Acc^{l_a}$表示客户端模型$l_a$在中心节点验证集的评价得分，$TotalAcc^{l_{1:A}}$表示所有客户端模型得分的和。


## FixMatch

![fix-match-process](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/federated%20learning/fl-2/fsel-fixmatch-process.png?raw=true)

​		在带标数据上进行弱数据增强，然后使用标准的交叉熵损失函数进行训练。

![fix-match-experiment](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/federated%20learning/fl-2/fsel-fixmatch-experiment.png?raw=true)

​		然后用模型性在弱数据增强的无标数据上进行预测![image-20201202144103762](/Users/benlihua/Library/Application Support/typora-user-images/image-20201202144103762.png)，选择其中超过阈值的样本生成one-hot人造标签，然后对强数据增强的无标签数据进行预测，最小化交叉熵损失函数。

![image-20201202142917010](/Users/benlihua/Library/Application Support/typora-user-images/image-20201202142917010.png)

​		最后的损失函数为，其中$\lambda_u$是固定的超参数
$$
\ell_s + \lambda_u\ell_u
$$
